In [2]:
import pandas as pd
import seaborn as sns
from imblearn.over_sampling import SMOTE
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [ ]:
def preprocessing(train_data):
    X = train_data.drop(columns=['Survived'])
    y = train_data['Survived']
    
    X['Age'] = X['Age'].fillna(X['Age'].mean())
    
    X = X.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
    
    imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    X.values[:] = imp.fit_transform(X)
    
    X = pd.get_dummies(X)
    
    scaler = MinMaxScaler()
    X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
    
    return X, y

In [7]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

df = pd.concat([train_data, test_data], sort=True).reset_index(drop=True)
y = train_data['SalePrice']

df = df.drop(columns=['Id'])

df['YrBltAndRemod'] = df['YearBuilt'] + df['YearRemodAdd']
df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']
df['Total_sqr_footage'] = df['BsmtFinSF1'] + df['BsmtFinSF2'] + df['1stFlrSF'] + df['2ndFlrSF']
df['Total_Bathrooms'] = df['FullBath'] + 0.5 * df['HalfBath'] + df['BsmtFullBath'] + 0.5 * df['BsmtHalfBath']
df['Total_porch_sf'] = df['OpenPorchSF'] + df['3SsnPorch'] + df['EnclosedPorch'] + df['ScreenPorch'] + df['WoodDeckSF']

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df.values[:] = imp.fit_transform(df)

df = df.fillna(df.mean())

df = pd.get_dummies(df)
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)

X = df.iloc[:len(y), :]
X_sub = df.iloc[len(y):, :]

In [10]:
reg = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, loss='huber', random_state=0 )

In [12]:
cross_val_score(reg, X, y, scoring='neg_mean_squared_error', cv=5)

array([-1.97010381e+08, -4.75340957e+08, -5.58173431e+08, -1.83172050e+08,
       -6.62942383e+08])